In [1]:
%matplotlib inline


Introduction to TorchScript
===========================

*James Reed (jamesreed@fb.com), Michael Suo (suo@fb.com)*, rev2

This tutorial is an introduction to TorchScript, an intermediate
representation of a PyTorch model (subclass of ``nn.Module``) that
can then be run in a high-performance environment such as C++.

In this tutorial we will cover:

1. The basics of model authoring in PyTorch, including:

-  Modules
-  Defining ``forward`` functions
-  Composing modules into a hierarchy of modules

2. Specific methods for converting PyTorch modules to TorchScript, our
   high-performance deployment runtime

-  Tracing an existing module
-  Using scripting to directly compile a module
-  How to compose both approaches
-  Saving and loading TorchScript modules

We hope that after you complete this tutorial, you will proceed to go through
`the follow-on tutorial <https://pytorch.org/tutorials/advanced/cpp_export.html>`_
which will walk you through an example of actually calling a TorchScript
model from C++.


In [2]:
import torch  # This is all you need to use both PyTorch and TorchScript!
print(torch.__version__)

1.9.0


Basics of PyTorch Model Authoring
---------------------------------

Let’s start out be defining a simple ``Module``. A ``Module`` is the
basic unit of composition in PyTorch. It contains:

1. A constructor, which prepares the module for invocation
2. A set of ``Parameters`` and sub-\ ``Modules``. These are initialized
   by the constructor and can be used by the module during invocation.
3. A ``forward`` function. This is the code that is run when the module
   is invoked.

Let’s examine a small example:




In [4]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()

    def forward(self, x, h):
        new_h = torch.tanh(x + h)
        return new_h, new_h

my_cell = MyCell()
x = torch.rand(3, 4)
h = torch.rand(3, 4)
print(my_cell(x, h))

(tensor([[0.8365, 0.9548, 0.3858, 0.5456],
        [0.8912, 0.6069, 0.4564, 0.8011],
        [0.1472, 0.9030, 0.2156, 0.7694]]), tensor([[0.8365, 0.9548, 0.3858, 0.5456],
        [0.8912, 0.6069, 0.4564, 0.8011],
        [0.1472, 0.9030, 0.2156, 0.7694]]))


So we’ve:

1. Created a class that subclasses ``torch.nn.Module``.
2. Defined a constructor. The constructor doesn’t do much, just calls
   the constructor for ``super``.
3. Defined a ``forward`` function, which takes two inputs and returns
   two outputs. The actual contents of the ``forward`` function are not
   really important, but it’s sort of a fake `RNN
   cell <https://colah.github.io/posts/2015-08-Understanding-LSTMs/>`__–that
   is–it’s a function that is applied on a loop.

We instantiated the module, and made ``x`` and ``h``, which are just 3x4
matrices of random values. Then we invoked the cell with
``my_cell(x, h)``. This in turn calls our ``forward`` function.

Let’s do something a little more interesting:




In [5]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.linear(x) + h)
        return new_h, new_h

my_cell = MyCell()
print(my_cell)
print(my_cell(x, h))

MyCell(
  (linear): Linear(in_features=4, out_features=4, bias=True)
)
(tensor([[ 0.7986,  0.8872,  0.4145, -0.0644],
        [ 0.8523,  0.3661,  0.2176,  0.4474],
        [ 0.5033,  0.8264,  0.6260,  0.6834]], grad_fn=<TanhBackward>), tensor([[ 0.7986,  0.8872,  0.4145, -0.0644],
        [ 0.8523,  0.3661,  0.2176,  0.4474],
        [ 0.5033,  0.8264,  0.6260,  0.6834]], grad_fn=<TanhBackward>))


We’ve redefined our module ``MyCell``, but this time we’ve added a
``self.linear`` attribute, and we invoke ``self.linear`` in the forward
function.

What exactly is happening here? ``torch.nn.Linear`` is a ``Module`` from
the PyTorch standard library. Just like ``MyCell``, it can be invoked
using the call syntax. We are building a hierarchy of ``Module``\ s.

``print`` on a ``Module`` will give a visual representation of the
``Module``\ ’s subclass hierarchy. In our example, we can see our
``Linear`` subclass and its parameters.

By composing ``Module``\ s in this way, we can succintly and readably
author models with reusable components.

You may have noticed ``grad_fn`` on the outputs. This is a detail of
PyTorch’s method of automatic differentiation, called
`autograd <https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html>`__.
In short, this system allows us to compute derivatives through
potentially complex programs. The design allows for a massive amount of
flexibility in model authoring.

Now let’s examine said flexibility:




In [ ]:
class MyDecisionGate(torch.nn.Module):
    def forward(self, x):
        if x.sum() > 0:
            return x
        else:
            return -x

class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.dg = MyDecisionGate()
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.dg(self.linear(x)) + h)
        return new_h, new_h

my_cell = MyCell()
print(my_cell)
print(my_cell(x, h))

We’ve once again redefined our MyCell class, but here we’ve defined
``MyDecisionGate``. This module utilizes **control flow**. Control flow
consists of things like loops and ``if``-statements.

Many frameworks take the approach of computing symbolic derivatives
given a full program representation. However, in PyTorch, we use a
gradient tape. We record operations as they occur, and replay them
backwards in computing derivatives. In this way, the framework does not
have to explicitly define derivatives for all constructs in the
language.

.. figure:: https://github.com/pytorch/pytorch/raw/master/docs/source/_static/img/dynamic_graph.gif
   :alt: How autograd works

   How autograd works




Basics of TorchScript
---------------------

Now let’s take our running example and see how we can apply TorchScript.

In short, TorchScript provides tools to capture the definition of your
model, even in light of the flexible and dynamic nature of PyTorch.
Let’s begin by examining what we call **tracing**.

Tracing ``Modules``
~~~~~~~~~~~~~~~~~~~




In [6]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.linear(x) + h)
        return new_h, new_h

my_cell = MyCell()
x, h = torch.rand(3, 4), torch.rand(3, 4)
traced_cell = torch.jit.trace(my_cell, (x, h))
print(traced_cell)
traced_cell(x, h)

MyCell(
  original_name=MyCell
  (linear): Linear(original_name=Linear)
)


(tensor([[ 0.1059,  0.5350,  0.5868,  0.3970],
         [-0.0311,  0.7782,  0.6097,  0.6892],
         [ 0.3636,  0.6238,  0.8861,  0.7523]], grad_fn=<TanhBackward>),
 tensor([[ 0.1059,  0.5350,  0.5868,  0.3970],
         [-0.0311,  0.7782,  0.6097,  0.6892],
         [ 0.3636,  0.6238,  0.8861,  0.7523]], grad_fn=<TanhBackward>))

We’ve rewinded a bit and taken the second version of our ``MyCell``
class. As before, we’ve instantiated it, but this time, we’ve called
``torch.jit.trace``, passed in the ``Module``, and passed in *example
inputs* the network might see.

What exactly has this done? It has invoked the ``Module``, recorded the
operations that occured when the ``Module`` was run, and created an
instance of ``torch.jit.ScriptModule`` (of which ``TracedModule`` is an
instance)

TorchScript records its definitions in an Intermediate Representation
(or IR), commonly referred to in Deep learning as a *graph*. We can
examine the graph with the ``.graph`` property:




In [7]:
print(traced_cell.graph)

graph(%self.1 : __torch__.MyCell,
      %x : Float(3, 4, strides=[4, 1], requires_grad=0, device=cpu),
      %h : Float(3, 4, strides=[4, 1], requires_grad=0, device=cpu)):
  %18 : __torch__.torch.nn.modules.linear.Linear = prim::GetAttr[name="linear"](%self.1)
  %20 : Tensor = prim::CallMethod[name="forward"](%18, %x)
  %11 : int = prim::Constant[value=1]() # /tmp/ipykernel_37437/260609686.py:7:0
  %12 : Float(3, 4, strides=[4, 1], requires_grad=1, device=cpu) = aten::add(%20, %h, %11) # /tmp/ipykernel_37437/260609686.py:7:0
  %13 : Float(3, 4, strides=[4, 1], requires_grad=1, device=cpu) = aten::tanh(%12) # /tmp/ipykernel_37437/260609686.py:7:0
  %14 : (Float(3, 4, strides=[4, 1], requires_grad=1, device=cpu), Float(3, 4, strides=[4, 1], requires_grad=1, device=cpu)) = prim::TupleConstruct(%13, %13)
  return (%14)



However, this is a very low-level representation and most of the
information contained in the graph is not useful for end users. Instead,
we can use the ``.code`` property to give a Python-syntax interpretation
of the code:




In [8]:
print(traced_cell.code)

def forward(self,
    x: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  _0 = torch.add((self.linear).forward(x, ), h)
  _1 = torch.tanh(_0)
  return (_1, _1)



So **why** did we do all this? There are several reasons:

1. TorchScript code can be invoked in its own interpreter, which is
   basically a restricted Python interpreter. This interpreter does not
   acquire the Global Interpreter Lock, and so many requests can be
   processed on the same instance simultaneously.
2. This format allows us to save the whole model to disk and load it
   into another environment, such as in a server written in a language
   other than Python
3. TorchScript gives us a representation in which we can do compiler
   optimizations on the code to provide more efficient execution
4. TorchScript allows us to interface with many backend/device runtimes
   that require a broader view of the program than individual operators.

We can see that invoking ``traced_cell`` produces the same results as
the Python module:




In [9]:
print(my_cell(x, h))
print(traced_cell(x, h))

(tensor([[ 0.1059,  0.5350,  0.5868,  0.3970],
        [-0.0311,  0.7782,  0.6097,  0.6892],
        [ 0.3636,  0.6238,  0.8861,  0.7523]], grad_fn=<TanhBackward>), tensor([[ 0.1059,  0.5350,  0.5868,  0.3970],
        [-0.0311,  0.7782,  0.6097,  0.6892],
        [ 0.3636,  0.6238,  0.8861,  0.7523]], grad_fn=<TanhBackward>))
(tensor([[ 0.1059,  0.5350,  0.5868,  0.3970],
        [-0.0311,  0.7782,  0.6097,  0.6892],
        [ 0.3636,  0.6238,  0.8861,  0.7523]], grad_fn=<TanhBackward>), tensor([[ 0.1059,  0.5350,  0.5868,  0.3970],
        [-0.0311,  0.7782,  0.6097,  0.6892],
        [ 0.3636,  0.6238,  0.8861,  0.7523]], grad_fn=<TanhBackward>))


Using Scripting to Convert Modules
----------------------------------

There’s a reason we used version two of our module, and not the one with
the control-flow-laden submodule. Let’s examine that now:




In [10]:
class MyDecisionGate(torch.nn.Module):
    def forward(self, x):
        if x.sum() > 0:
            return x
        else:
            return -x

class MyCell(torch.nn.Module):
    def __init__(self, dg):
        super(MyCell, self).__init__()
        self.dg = dg
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.dg(self.linear(x)) + h)
        return new_h, new_h

my_cell = MyCell(MyDecisionGate())
traced_cell = torch.jit.trace(my_cell, (x, h))

print(traced_cell.dg.code)
print(traced_cell.code)

def forward(self,
    argument_1: Tensor) -> Tensor:
  return torch.neg(argument_1)

def forward(self,
    x: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  _0 = (self.dg).forward((self.linear).forward(x, ), )
  _1 = torch.tanh(torch.add(_0, h))
  return (_1, _1)



/tmp/ipykernel_37437/4234398751.py:3: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x.sum() > 0:


Looking at the ``.code`` output, we can see that the ``if-else`` branch
is nowhere to be found! Why? Tracing does exactly what we said it would:
run the code, record the operations *that happen* and construct a
ScriptModule that does exactly that. Unfortunately, things like control
flow are erased.

How can we faithfully represent this module in TorchScript? We provide a
**script compiler**, which does direct analysis of your Python source
code to transform it into TorchScript. Let’s convert ``MyDecisionGate``
using the script compiler:




In [11]:
scripted_gate = torch.jit.script(MyDecisionGate())

my_cell = MyCell(scripted_gate)
scripted_cell = torch.jit.script(my_cell)

print(scripted_gate.code)
print(scripted_cell.code)

def forward(self,
    x: Tensor) -> Tensor:
  if bool(torch.gt(torch.sum(x), 0)):
    _0 = x
  else:
    _0 = torch.neg(x)
  return _0

def forward(self,
    x: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  _0 = (self.dg).forward((self.linear).forward(x, ), )
  new_h = torch.tanh(torch.add(_0, h))
  return (new_h, new_h)



Hooray! We’ve now faithfully captured the behavior of our program in
TorchScript. Let’s now try running the program:




In [12]:
# New inputs
x, h = torch.rand(3, 4), torch.rand(3, 4)
traced_cell(x, h)

(tensor([[0.1444, 0.6312, 0.6598, 0.5327],
         [0.8774, 0.8696, 0.0562, 0.8709],
         [0.7141, 0.8917, 0.6074, 0.7720]], grad_fn=<TanhBackward>),
 tensor([[0.1444, 0.6312, 0.6598, 0.5327],
         [0.8774, 0.8696, 0.0562, 0.8709],
         [0.7141, 0.8917, 0.6074, 0.7720]], grad_fn=<TanhBackward>))

Mixing Scripting and Tracing
~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Some situations call for using tracing rather than scripting (e.g. a
module has many architectural decisions that are made based on constant
Python values that we would like to not appear in TorchScript). In this
case, scripting can be composed with tracing: ``torch.jit.script`` will
inline the code for a traced module, and tracing will inline the code
for a scripted module.

An example of the first case:




In [13]:
class MyRNNLoop(torch.nn.Module):
    def __init__(self):
        super(MyRNNLoop, self).__init__()
        self.cell = torch.jit.trace(MyCell(scripted_gate), (x, h))

    def forward(self, xs):
        h, y = torch.zeros(3, 4), torch.zeros(3, 4)
        for i in range(xs.size(0)):
            y, h = self.cell(xs[i], h)
        return y, h

rnn_loop = torch.jit.script(MyRNNLoop())
print(rnn_loop.code)

def forward(self,
    xs: Tensor) -> Tuple[Tensor, Tensor]:
  h = torch.zeros([3, 4])
  y = torch.zeros([3, 4])
  y0 = y
  h0 = h
  for i in range(torch.size(xs, 0)):
    _0 = (self.cell).forward(torch.select(xs, 0, i), h0, )
    y1, h1, = _0
    y0, h0 = y1, h1
  return (y0, h0)



And an example of the second case:




In [14]:
class WrapRNN(torch.nn.Module):
    def __init__(self):
        super(WrapRNN, self).__init__()
        self.loop = torch.jit.script(MyRNNLoop())

    def forward(self, xs):
        y, h = self.loop(xs)
        return torch.relu(y)

traced = torch.jit.trace(WrapRNN(), (torch.rand(10, 3, 4)))
print(traced.code)

def forward(self,
    xs: Tensor) -> Tensor:
  _0, y, = (self.loop).forward(xs, )
  return torch.relu(y)



This way, scripting and tracing can be used when the situation calls for
each of them and used together.

Saving and Loading models
-------------------------

We provide APIs to save and load TorchScript modules to/from disk in an
archive format. This format includes code, parameters, attributes, and
debug information, meaning that the archive is a freestanding
representation of the model that can be loaded in an entirely separate
process. Let’s save and load our wrapped RNN module:




In [15]:
traced.save('wrapped_rnn.pt')

loaded = torch.jit.load('wrapped_rnn.pt')

print(loaded)
print(loaded.code)

RecursiveScriptModule(
  original_name=WrapRNN
  (loop): RecursiveScriptModule(
    original_name=MyRNNLoop
    (cell): RecursiveScriptModule(
      original_name=MyCell
      (dg): RecursiveScriptModule(original_name=MyDecisionGate)
      (linear): RecursiveScriptModule(original_name=Linear)
    )
  )
)
def forward(self,
    xs: Tensor) -> Tensor:
  _0, y, = (self.loop).forward(xs, )
  return torch.relu(y)



As you can see, serialization preserves the module hierarchy and the
code we’ve been examining throughout. The model can also be loaded, for
example, `into
C++ <https://pytorch.org/tutorials/advanced/cpp_export.html>`__ for
python-free execution.

Further Reading
~~~~~~~~~~~~~~~

We’ve completed our tutorial! For a more involved demonstration, check
out the NeurIPS demo for converting machine translation models using
TorchScript:
https://colab.research.google.com/drive/1HiICg6jRkBnr5hvK2-VnMi88Vi9pUzEJ




import torch
import torchvision

# An instance of your model.
model = torchvision.models.resnet18()

# An example input you would normally provide to your model's forward() method.
example = torch.rand(1, 3, 224, 224)

# Use torch.jit.trace to generate a torch.jit.ScriptModule via tracing.
traced_script_module = torch.jit.trace(model, example)

In [16]:
import torch
import torchvision

# An instance of your model.
model = torchvision.models.resnet18()

# An example input you would normally provide to your model's forward() method.
example = torch.rand(1, 3, 224, 224)

# Use torch.jit.trace to generate a torch.jit.ScriptModule via tracing.
traced_script_module = torch.jit.trace(model, example)

/home/ayoola/miniconda3/envs/pytorch/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448238472/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [19]:
print(traced_script_module.code)"""  """

def forward(self,
    x: Tensor) -> Tensor:
  _0 = self.fc
  _1 = self.avgpool
  _2 = self.layer4
  _3 = self.layer3
  _4 = self.layer2
  _5 = self.layer1
  _6 = self.maxpool
  _7 = self.relu
  _8 = (self.bn1).forward((self.conv1).forward(x, ), )
  _9 = (_5).forward((_6).forward((_7).forward(_8, ), ), )
  _10 = (_2).forward((_3).forward((_4).forward(_9, ), ), )
  input = torch.flatten((_1).forward(_10, ), 1)
  return (_0).forward(input, )



In [20]:
class MyModule(torch.nn.Module):
    def __init__(self, N, M):
        super(MyModule, self).__init__()
        self.weight = torch.nn.Parameter(torch.rand(N, M))

    def forward(self, input):
        if input.sum() > 0:
          output = self.weight.mv(input)
        else:
          output = self.weight + input
        return output

my_module = MyModule(10,20)
sm = torch.jit.script(my_module)

In [21]:
print(sm.code())

TypeError: 'str' object is not callable

In [28]:
traced_script_module.save('ayo.pt')

In [27]:
sm.save()

TypeError: save() missing 1 required positional argument: 'f'

In [25]:
sm.save('bree.pt')

In [1]:
import torch, torchvision
model = torchvision.models.resnet18(pretrained=True)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/ayoola/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100.0%


In [2]:
prediction = model(data) # forward pass

/home/ayoola/miniconda3/envs/pytorch/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448238472/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [3]:
loss = (prediction - labels).sum()
loss.backward() # backward pass

In [4]:
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

In [5]:
optim.step() #gradient descent

In [6]:
import torch

a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)

In [7]:
Q = 3*a**3 - b**2

In [8]:
external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad)

In [9]:
# check if collected gradients are correct
print(9*a**2 == a.grad)
print(-2*b == b.grad)

tensor([True, True])
tensor([True, True])


In [10]:
x = torch.rand(5, 5)
y = torch.rand(5, 5)
z = torch.rand((5, 5), requires_grad=True)

a = x + y
print(f"Does `a` require gradients? : {a.requires_grad}")
b = x + z
print(f"Does `b` require gradients?: {b.requires_grad}")

Does `a` require gradients? : False
Does `b` require gradients?: True


In [11]:
from torch import nn, optim

model = torchvision.models.resnet18(pretrained=True)

# Freeze all the parameters in the network
for param in model.parameters():
    param.requires_grad = False

In [12]:
model.fc = nn.Linear(512, 10)

In [ ]:
# Optimize only the classifier
optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)